In [1]:
import dash
from dash import dcc, html, Input, Output, State
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import numpy as np
from dash.exceptions import PreventUpdate
import locale

# Set locale for currency formatting
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

# Initialize Dash app
app = dash.Dash(__name__, 
                meta_tags=[{"name": "viewport", "content": "width=device-width, initial-scale=1"}])
server = app.server
app.title = "Landslide Risk Assessment Dashboard"

# Sample data for different regions (same as in TypeScript)
region_data = {
    'Region A': {
        'populationDensity': 1200,
        'infrastructureValue': 45000000,
        'annualIncome': 38000,
        'landValue': 2500,
        'historicalEvents': 12,
        'slope': 32,
        'vegetation': 'Low',
        'rainfall': 'High'
    },
    'Region B': {
        'populationDensity': 850,
        'infrastructureValue': 32000000,
        'annualIncome': 42000,
        'landValue': 3200,
        'historicalEvents': 8,
        'slope': 28,
        'vegetation': 'Medium',
        'rainfall': 'Medium'
    },
    'Region C': {
        'populationDensity': 1800,
        'infrastructureValue': 67000000,
        'annualIncome': 35000,
        'landValue': 4100,
        'historicalEvents': 17,
        'slope': 38,
        'vegetation': 'Very Low',
        'rainfall': 'Very High'
    }
}

# Color schemes
COLORS = ['#0088FE', '#00C49F', '#FFBB28', '#FF8042']

# Function to format currency
def format_currency(value):
    return locale.currency(value, grouping=True)

# Function to calculate Value at Risk (same logic as TypeScript version)
def calculate_var(confidence, region, horizon, budget):
    region_info = region_data[region]
    
    # Risk factors based on region characteristics
    population_risk = region_info['populationDensity'] / 1000
    slope_risk = region_info['slope'] / 40
    
    vegetation_factors = {'Very Low': 0.9, 'Low': 0.7, 'Medium': 0.5, 'High': 0.3, 'Very High': 0.1}
    rainfall_factors = {'Very Low': 0.1, 'Low': 0.3, 'Medium': 0.5, 'High': 0.7, 'Very High': 0.9}
    
    vegetation_factor = vegetation_factors[region_info['vegetation']]
    rainfall_factor = rainfall_factors[region_info['rainfall']]
    historical_factor = min(1, region_info['historicalEvents'] / 20)
    
    # Combined risk score (0-1)
    base_risk_score = (population_risk * 0.2 + slope_risk * 0.25 + vegetation_factor * 0.2 + 
                        rainfall_factor * 0.25 + historical_factor * 0.1)
    
    # Adjust risk based on confidence level
    confidence_adjustment = (confidence - 50) / 50
    adjusted_risk = base_risk_score * (1 + confidence_adjustment * 0.5)
    
    # Scale by time horizon (using a sublinear function to account for mitigation measures over time)
    time_adjusted_risk = adjusted_risk * (horizon ** 0.7)
    
    # Impact calculations
    casualties = round(region_info['populationDensity'] * time_adjusted_risk * 0.01 * horizon / 10)
    infrastructure_loss = region_info['infrastructureValue'] * time_adjusted_risk * 0.2
    land_area_lost = 1000 * time_adjusted_risk  # in hectares
    land_value_loss = land_area_lost * region_info['landValue']
    income_loss = region_info['annualIncome'] * casualties * 5  # Assuming 5 years of income loss per casualty
    displacement_count = round(region_info['populationDensity'] * time_adjusted_risk * 0.1)
    displacement_cost = displacement_count * 25000  # Cost per displaced person
    
    total_economic_impact = infrastructure_loss + land_value_loss + income_loss + displacement_cost
    
    # Calculate how mitigation budget would reduce impacts
    mitigation_effectiveness = min(0.8, budget / (total_economic_impact * 0.5))
    mitigated_total_impact = total_economic_impact * (1 - mitigation_effectiveness)
    mitigated_casualties = casualties * (1 - mitigation_effectiveness * 0.9)  # Higher effectiveness for human safety
    
    # Return period calculation
    probability = min(0.99, time_adjusted_risk)
    return_period = round(1 / probability)
    
    # Cost breakdown for pie chart
    cost_breakdown = [
        {'name': 'Infrastructure', 'value': infrastructure_loss},
        {'name': 'Land Value', 'value': land_value_loss},
        {'name': 'Income Loss', 'value': income_loss},
        {'name': 'Displacement', 'value': displacement_cost}
    ]
    
    # Calculate mitigation ROI
    mitigation_roi = (total_economic_impact - mitigated_total_impact) / budget if budget > 0 else 0
    
    return {
        'probability': round(probability * 100, 1),
        'casualties': round(casualties),
        'mitigated_casualties': round(mitigated_casualties),
        'infrastructure_loss': infrastructure_loss,
        'land_area_lost': round(land_area_lost, 1),
        'land_value_loss': land_value_loss,
        'income_loss': income_loss,
        'displacement_count': displacement_count,
        'displacement_cost': displacement_cost,
        'total_economic_impact': total_economic_impact,
        'mitigated_economic_impact': mitigated_total_impact,
        'return_period': return_period,
        'cost_breakdown': cost_breakdown,
        'mitigation_roi': mitigation_roi
    }

# Layout of the dashboard
app.layout = html.Div(
    className="container",
    style={'backgroundColor': '#f5f5f5', 'padding': '20px'},
    children=[
        html.H1("Landslide Risk Assessment Dashboard", className="title"),
        
        # Controls section
        html.Div(
            className="controls-container",
            style={'display': 'grid', 'gridTemplateColumns': 'repeat(auto-fit, minmax(250px, 1fr))', 'gap': '20px', 'marginBottom': '30px'},
            children=[
                # Confidence level control
                html.Div(
                    className="control-card",
                    style={'backgroundColor': 'white', 'padding': '15px', 'borderRadius': '5px', 'boxShadow': '0 2px 4px rgba(0,0,0,0.1)'},
                    children=[
                        html.Label("Risk Confidence Level:", className="control-label"),
                        html.Div(id="confidence-value", style={'fontWeight': 'bold', 'marginBottom': '10px'}),
                        dcc.Slider(
                            id="confidence-slider",
                            min=50,
                            max=99,
                            value=95,
                            marks={50: '50%', 75: '75%', 95: '95%', 99: '99%'},
                            step=1
                        ),
                        html.P("Higher values represent more conservative risk estimates", 
                              style={'fontSize': '12px', 'color': '#666', 'marginTop': '10px'})
                    ]
                ),
                
                # Region selection
                html.Div(
                    className="control-card",
                    style={'backgroundColor': 'white', 'padding': '15px', 'borderRadius': '5px', 'boxShadow': '0 2px 4px rgba(0,0,0,0.1)'},
                    children=[
                        html.Label("Region:", className="control-label"),
                        dcc.Dropdown(
                            id="region-dropdown",
                            options=[{"label": region, "value": region} for region in region_data.keys()],
                            value="Region A",
                            clearable=False
                        )
                    ]
                ),
                
                # Time horizon control
                html.Div(
                    className="control-card",
                    style={'backgroundColor': 'white', 'padding': '15px', 'borderRadius': '5px', 'boxShadow': '0 2px 4px rgba(0,0,0,0.1)'},
                    children=[
                        html.Label("Time Horizon:", className="control-label"),
                        html.Div(id="time-value", style={'fontWeight': 'bold', 'marginBottom': '10px'}),
                        dcc.Slider(
                            id="time-slider",
                            min=1,
                            max=50,
                            value=10,
                            marks={1: '1 yr', 10: '10 yrs', 25: '25 yrs', 50: '50 yrs'},
                            step=1
                        )
                    ]
                ),
                
                # Mitigation budget control
                html.Div(
                    className="control-card",
                    style={'backgroundColor': 'white', 'padding': '15px', 'borderRadius': '5px', 'boxShadow': '0 2px 4px rgba(0,0,0,0.1)'},
                    children=[
                        html.Label("Mitigation Budget:", className="control-label"),
                        html.Div(id="budget-value", style={'fontWeight': 'bold', 'marginBottom': '10px'}),
                        dcc.Slider(
                            id="budget-slider",
                            min=0,
                            max=10000000,
                            value=5000000,
                            marks={0: '$0', 2500000: '$2.5M', 5000000: '$5M', 7500000: '$7.5M', 10000000: '$10M'},
                            step=500000
                        )
                    ]
                )
            ]
        ),
        
        # Key metrics section
        html.Div(
            id="metrics-container",
            style={'display': 'grid', 'gridTemplateColumns': 'repeat(auto-fit, minmax(250px, 1fr))', 'gap': '20px', 'marginBottom': '30px'}
        ),
        
        # Main charts
        html.Div(
            className="charts-container",
            style={'display': 'grid', 'gridTemplateColumns': 'repeat(auto-fit, minmax(500px, 1fr))', 'gap': '20px', 'marginBottom': '30px'},
            children=[
                # VaR Curve
                html.Div(
                    className="chart-card",
                    style={'backgroundColor': 'white', 'padding': '15px', 'borderRadius': '5px', 'boxShadow': '0 2px 4px rgba(0,0,0,0.1)'},
                    children=[
                        html.H3("Value at Risk by Confidence Level", className="chart-title"),
                        dcc.Graph(id="var-curve-chart")
                    ]
                ),
                
                # Cost Breakdown
                html.Div(
                    className="chart-card",
                    style={'backgroundColor': 'white', 'padding': '15px', 'borderRadius': '5px', 'boxShadow': '0 2px 4px rgba(0,0,0,0.1)'},
                    children=[
                        html.H3("Impact Breakdown", className="chart-title"),
                        dcc.Graph(id="cost-breakdown-chart")
                    ]
                )
            ]
        ),
        
        # Mitigation Analysis
        html.Div(
            className="chart-card",
            style={'backgroundColor': 'white', 'padding': '15px', 'borderRadius': '5px', 'boxShadow': '0 2px 4px rgba(0,0,0,0.1)', 'marginBottom': '30px'},
            children=[
                html.H3("Mitigation Budget Analysis", className="chart-title"),
                dcc.Graph(id="mitigation-chart")
            ]
        ),
        
        # Region Details
        html.Div(
            id="region-details",
            className="details-card",
            style={'backgroundColor': 'white', 'padding': '15px', 'borderRadius': '5px', 'boxShadow': '0 2px 4px rgba(0,0,0,0.1)', 'marginBottom': '30px'}
        ),
        
        # Explanation section
        html.Div(
            className="explanation-card",
            style={'backgroundColor': 'white', 'padding': '15px', 'borderRadius': '5px', 'boxShadow': '0 2px 4px rgba(0,0,0,0.1)'},
            children=[
                html.H3("About This Dashboard", className="section-title"),
                html.P(
                    """This dashboard applies Value at Risk (VaR) methodology to landslide risk assessment. 
                    By adjusting the confidence level, you're specifying how conservative your risk estimate should be. 
                    For example, a 95% confidence level means there's a 95% probability that losses will not exceed the calculated VaR amount.""",
                    className="explanation-text"
                ),
                html.Div(
                    style={'display': 'grid', 'gridTemplateColumns': 'repeat(auto-fit, minmax(300px, 1fr))', 'gap': '20px', 'marginTop': '20px'},
                    children=[
                        html.Div(
                            children=[
                                html.H4("Key Metrics Explained", className="subsection-title"),
                                html.Ul(
                                    children=[
                                        #html.Li(html.Strong("Value at Risk (VaR): ") + "The maximum economic loss expected with the specified confidence level"),
                                        #html.Li(html.Strong("Return Period: ") + "How frequently a landslide event of this magnitude is expected to occur"),
                                        #html.Li(html.Strong("Mitigation ROI: ") + "Return on investment for risk reduction measures")
                                        html.Li(("Value at Risk (VaR): ") + "The maximum economic loss expected with the specified confidence level"),
                                        html.Li(("Return Period: ") + "How frequently a landslide event of this magnitude is expected to occur"),
                                        html.Li(("Mitigation ROI: ") + "Return on investment for risk reduction measures")
                                    ],
                                    className="explanation-list"
                                )
                            ]
                        ),
                        html.Div(
                            children=[
                                html.H4("Decision Support", className="subsection-title"),
                                html.Ul(
                                    children=[
                                        html.Li("Compare different regions to prioritize interventions"),
                                        html.Li("Adjust the mitigation budget to find optimal spending levels"),
                                        html.Li("Test different time horizons for short and long-term planning"),
                                        html.Li("Analyze cost breakdowns to target specific vulnerability factors")
                                    ],
                                    className="explanation-list"
                                )
                            ]
                        )
                    ]
                )
            ]
        )
    ]
)

# Callbacks to update UI elements

# Update slider value displays
@app.callback(
    Output("confidence-value", "children"),
    [Input("confidence-slider", "value")]
)
def update_confidence_value(value):
    return f"{value}%"

@app.callback(
    Output("time-value", "children"),
    [Input("time-slider", "value")]
)
def update_time_value(value):
    return f"{value} Years"

@app.callback(
    Output("budget-value", "children"),
    [Input("budget-slider", "value")]
)
def update_budget_value(value):
    return format_currency(value)

# Update metrics
@app.callback(
    Output("metrics-container", "children"),
    [
        Input("confidence-slider", "value"),
        Input("region-dropdown", "value"),
        Input("time-slider", "value"),
        Input("budget-slider", "value")
    ]
)
def update_metrics(confidence, region, time_horizon, budget):
    var_results = calculate_var(confidence, region, time_horizon, budget)
    
    return [
        # Value at Risk
        html.Div(
            className="metric-card",
            style={'backgroundColor': 'white', 'padding': '15px', 'borderRadius': '5px', 'boxShadow': '0 2px 4px rgba(0,0,0,0.1)'},
            children=[
                html.H3("Value at Risk (VaR)", className="metric-title"),
                html.P(format_currency(var_results['total_economic_impact']), className="metric-value", style={'color': '#dc2626', 'fontSize': '24px', 'fontWeight': 'bold'}),
                html.P(f"at {confidence}% confidence over {time_horizon} years", className="metric-subtitle"),
                html.P(
                    f"Probability: {var_results['probability']}% | Return period: 1 in {var_results['return_period']} years",
                    className="metric-detail"
                )
            ]
        ),
        
        # Casualties
        html.Div(
            className="metric-card",
            style={'backgroundColor': 'white', 'padding': '15px', 'borderRadius': '5px', 'boxShadow': '0 2px 4px rgba(0,0,0,0.1)'},
            children=[
                html.H3("Estimated Casualties", className="metric-title"),
                html.P(str(var_results['casualties']), className="metric-value", style={'color': '#dc2626', 'fontSize': '24px', 'fontWeight': 'bold'}),
                html.P(f"With mitigation: {var_results['mitigated_casualties']}", className="metric-subtitle"),
                html.P(f"Displaced persons: {var_results['displacement_count']:,}", className="metric-detail")
            ]
        ),
        
        # Land Impact
        html.Div(
            className="metric-card",
            style={'backgroundColor': 'white', 'padding': '15px', 'borderRadius': '5px', 'boxShadow': '0 2px 4px rgba(0,0,0,0.1)'},
            children=[
                html.H3("Land Impact", className="metric-title"),
                html.P(f"{var_results['land_area_lost']} ha", className="metric-value", style={'color': '#d97706', 'fontSize': '24px', 'fontWeight': 'bold'}),
                html.P(f"Value: {format_currency(var_results['land_value_loss'])}", className="metric-subtitle")
            ]
        ),
        
        # Mitigation ROI
        html.Div(
            className="metric-card",
            style={'backgroundColor': 'white', 'padding': '15px', 'borderRadius': '5px', 'boxShadow': '0 2px 4px rgba(0,0,0,0.1)'},
            children=[
                html.H3("Mitigation ROI", className="metric-title"),
                html.P(f"{var_results['mitigation_roi']:.2f}x", className="metric-value", style={'color': '#059669', 'fontSize': '24px', 'fontWeight': 'bold'}),
                html.P(f"Risk reduced by {int(var_results['mitigation_roi'] * 100)}%", className="metric-subtitle")
            ]
        )
    ]

# Update VaR curve chart
@app.callback(
    Output("var-curve-chart", "figure"),
    [
        Input("region-dropdown", "value"),
        Input("time-slider", "value"),
        Input("budget-slider", "value")
    ]
)
def update_var_curve(region, time_horizon, budget):
    # Generate data for different confidence levels
    conf_levels = list(range(50, 100, 5))
    var_curve_data = []
    
    for conf in conf_levels:
        result = calculate_var(conf, region, time_horizon, budget)
        var_curve_data.append({
            'confidence_level': conf,
            'economic_impact': result['total_economic_impact'] / 1000000
        })
    
    df = pd.DataFrame(var_curve_data)
    
    fig = px.line(
        df, 
        x='confidence_level', 
        y='economic_impact',
        labels={
            'confidence_level': 'Confidence Level (%)',
            'economic_impact': 'Impact (Millions $)'
        }
    )
    
    fig.update_traces(line=dict(width=3, color='#8884d8'))
    fig.update_layout(
        margin=dict(l=40, r=40, t=20, b=40),
        xaxis_title='Confidence Level (%)',
        yaxis_title='Impact (Millions $)',
        font=dict(family='Arial, sans-serif', size=12),
        plot_bgcolor='white',
        hovermode='closest'
    )
    fig.update_xaxes(showgrid=True, gridwidth=0.5, gridcolor='LightGray', griddash='dash')
    fig.update_yaxes(showgrid=True, gridwidth=0.5, gridcolor='LightGray', griddash='dash')
    
    return fig

# Update cost breakdown chart
@app.callback(
    Output("cost-breakdown-chart", "figure"),
    [
        Input("confidence-slider", "value"),
        Input("region-dropdown", "value"),
        Input("time-slider", "value"),
        Input("budget-slider", "value")
    ]
)
def update_cost_breakdown(confidence, region, time_horizon, budget):
    var_results = calculate_var(confidence, region, time_horizon, budget)
    
    # Create DataFrame for cost breakdown
    breakdown_data = pd.DataFrame(var_results['cost_breakdown'])
    
    # Create pie chart
    fig = px.pie(
        breakdown_data, 
        values='value', 
        names='name',
        color_discrete_sequence=COLORS,
        hole=0.4
    )
    
    fig.update_traces(
        textposition='inside',
        textinfo='percent+label', 
        marker=dict(line=dict(color='white', width=2))
    )
    
    fig.update_layout(
        margin=dict(l=20, r=20, t=30, b=20),
        legend_title='Cost Category',
        font=dict(family='Arial, sans-serif'),
        annotations=[dict(
            text=f'Total: {format_currency(var_results["total_economic_impact"])}',
            showarrow=False,
            font_size=12
        )]
    )
    
    return fig

# Update mitigation analysis chart
@app.callback(
    Output("mitigation-chart", "figure"),
    [
        Input("confidence-slider", "value"),
        Input("region-dropdown", "value"),
        Input("time-slider", "value")
    ]
)
def update_mitigation_chart(confidence, region, time_horizon):
    # Generate data for different budget levels
    budgets = list(range(0, 10000001, 1000000))
    mitigation_data = []
    
    for budget in budgets:
        result = calculate_var(confidence, region, time_horizon, budget)
        mitigation_data.append({
            'budget': budget / 1000000,  # Convert to millions
            'impact': result['mitigated_economic_impact'] / 1000000,  # Convert to millions
            'roi': result['mitigation_roi'] if budget > 0 else 0
        })
    
    df = pd.DataFrame(mitigation_data)
    
    # Create the figure with two y-axes
    fig = go.Figure()
    
    # Add residual impact line
    fig.add_trace(
        go.Scatter(
            x=df['budget'],
            y=df['impact'],
            mode='lines',
            name='Residual Economic Impact',
            line=dict(color='#ff7300', width=3)
        )
    )
    
    # Add ROI line on secondary y-axis
    fig.add_trace(
        go.Scatter(
            x=df['budget'],
            y=df['roi'],
            mode='lines',
            name='Return on Investment',
            line=dict(color='#387908', width=3),
            yaxis='y2'
        )
    )
    
    # Update layout with second y-axis
    fig.update_layout(
        xaxis=dict(title='Mitigation Budget (Millions $)'),
        yaxis=dict(title='Residual Impact (Millions $)', side='left'),
        yaxis2=dict(title='ROI', overlaying='y', side='right'),
        #legend=dict(x=0.01, y=0.99, orientation='h'),
        legend=dict(x=0.85, y=0.90, orientation='v'),
        margin=dict(l=60, r=60, t=30, b=50),
        hovermode='closest',
        plot_bgcolor='white'
    )
    fig.update_xaxes(showgrid=True, gridwidth=0.5, gridcolor='LightGray', griddash='dash')
    fig.update_yaxes(showgrid=True, gridwidth=0.5, gridcolor='LightGray', griddash='dash')
    
    return fig

# Update region details
@app.callback(
    Output("region-details", "children"),
    [Input("region-dropdown", "value")]
)
def update_region_details(region):
    region_info = region_data[region]
    
    return [
        html.H3(f"Region Details: {region}", className="section-title"),
        html.Div(
            style={'display': 'grid', 'gridTemplateColumns': 'repeat(auto-fit, minmax(200px, 1fr))', 'gap': '20px', 'marginTop': '15px'},
            children=[
                html.Div([
                    html.H4("Population Density", className="detail-title"),
                    html.P(f"{region_info['populationDensity']} per km²", className="detail-value")
                ]),
                html.Div([
                    html.H4("Infrastructure Value", className="detail-title"),
                    html.P(format_currency(region_info['infrastructureValue']), className="detail-value")
                ]),
                html.Div([
                    html.H4("Annual Income per Capita", className="detail-title"),
                    html.P(format_currency(region_info['annualIncome']), className="detail-value")
                ]),
                html.Div([
                    html.H4("Land Value", className="detail-title"),
                    html.P(f"{format_currency(region_info['landValue'])}/ha", className="detail-value")
                ]),
                html.Div([
                    html.H4("Historical Events", className="detail-title"),
                    html.P(f"{region_info['historicalEvents']} incidents", className="detail-value")
                ]),
                html.Div([
                    html.H4("Average Slope", className="detail-title"),
                    html.P(f"{region_info['slope']}°", className="detail-value")
                ]),
                html.Div([
                    html.H4("Vegetation Cover", className="detail-title"),
                    html.P(region_info['vegetation'], className="detail-value")
                ]),
                html.Div([
                    html.H4("Rainfall Intensity", className="detail-title"),
                    html.P(region_info['rainfall'], className="detail-value")
                ])
            ]
        )
    ]

# Run the app
#if __name__ == '__main__':
#    app.run_server(debug=True)

In [2]:
app.run_server(debug=True)